In [28]:
import pandas as pd
import json

* Read the results of the GA executions on the complete instances 

* To present the execution results for the operators on the sample, read file `sample_results_real.json` 

In [29]:
with open("ga_results_eaai.json", "r") as file:
    operator_results = json.load(file)

* Compute averages per trial, max and min trial for each instance and each operator

In [30]:
operator_stats = {}

for instance, instance_data in operator_results.items():
    ga_methods = instance_data.get("method:", {}).get("GA", {})

    operator_stats[instance] = {}

    # Find best fitness across all trials with the same stages and orders
    instance_prefix = instance.split('-I_')[0]
    all_makespan_same_group = [
        1 / metrics["fitness"]
        for inst, inst_data in operator_results.items()
        if inst.startswith(instance_prefix)
        for op_data in inst_data.get("method:", {}).get("GA", {}).values()
        for trial_set2 in op_data.get("detailed_trial_results", [])
        for metrics in trial_set2["trial_results"].values()
    ]

    best_makespan = min(all_makespan_same_group)
    for operator, operator_data in ga_methods.items():
        for trial_set in operator_data.get("detailed_trial_results", []):
            trial_results = trial_set["trial_results"]
            fitness_values = []
            trial_metrics = []
            for trial_name, metrics in trial_results.items():
                fitness_values.append(metrics["fitness"])
                trial_metrics.append((trial_name, metrics))

            # compute average metrics
            avg_metrics = {}
            if trial_metrics:
                keys = trial_metrics[0][1].keys()
                for key in keys:
                    if key.endswith("chromosome"):
                        continue
                    avg_metrics[key] = sum(m[key] for _, m in trial_metrics) / len(trial_metrics)
                # compute average RPD
                rpd_values = [((1 / m["fitness"]) - best_makespan) / best_makespan for _, m in trial_metrics]
                avg_metrics["RPD"] = sum(rpd_values) / len(rpd_values) if rpd_values else None
            # find max and min fitness trials
            max_idx = fitness_values.index(max(fitness_values))
            min_idx = fitness_values.index(min(fitness_values))
            max_trial = trial_metrics[max_idx][1]
            min_trial = trial_metrics[min_idx][1]
            operator_stats[instance][operator] = {
                "average": avg_metrics,
                "max_fitness_trial": max_trial,
                "min_fitness_trial": min_trial
            }

* Save the detailed results

In [31]:
if operator_stats != {}:
    with open("operators.json", 'w') as file:
        json.dump(operator_stats, file, indent=4)

* Create a dataframe with the averages 

In [32]:
data = {}
for instance, ops in operator_stats.items():
    row = {}
    for operator, stats in ops.items():
        for metric, value in stats["average"].items():
            row[f"{operator}_{metric}_average"] = value
    data[instance] = row

df_operator_avg = pd.DataFrame.from_dict(data, orient="index")

* Group the results per stages-orders categories

In [33]:
grouped = df_operator_avg.groupby(
    df_operator_avg.index.to_series().str.replace(r'-I_\d+', '', regex=True)
)

* Print the averages per stages-orders categories

In [34]:
grouped.mean().T

,S_3-Ord_10-WIP_0,S_3-Ord_30-WIP_0,S_3-Ord_50-WIP_0,S_6-Ord_10-WIP_0,S_6-Ord_30-WIP_0,S_6-Ord_50-WIP_0,S_9-Ord_10-WIP_0,S_9-Ord_30-WIP_0,S_9-Ord_50-WIP_0
swap_fitness_average,0.011922,0.002103,0.003488,0.008242,0.003636,0.001836,0.005496,0.002621,0.002038
swap_time_found_average,60.919310,51.541257,635.079202,63.459572,441.134881,926.177250,73.407896,777.975271,2268.227733
swap_total_time_average,103.521517,511.398039,815.437473,145.499152,564.115404,1469.711723,156.733120,892.358886,2518.904566
swap_generation_found_average,48.777778,10.111111,72.000000,39.444444,73.555556,60.555556,44.555556,81.888889,91.555556
swap_RPD_average,0.298507,0.084091,0.002331,0.408187,0.338384,0.571759,0.046616,0.438534,0.390794
